In [1]:
from cryptography.fernet import Fernet
import subprocess
import os
import requests

import getpass
username = getpass.getuser()

In [2]:
def generate_key():
    # Generate a key
    key = Fernet.generate_key()
    # Save the key to a file
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

def load_key():
    # Load the key from the current directory
    return open("secret.key", "rb").read()

def encrypt_token(token):
    key = load_key()
    f = Fernet(key)
    encrypted_token = f.encrypt(token.encode())
    with open("encrypted_git_token.txt", "wb") as encrypted_file:
        encrypted_file.write(encrypted_token)

In [3]:
# token = ''
# generate_key()
# encrypt_token(token)

In [4]:
from cryptography.fernet import Fernet
import os

def load_key():
    # Load the key from the current directory
    return open("secret.key", "rb").read()

def decrypt_token():
    key = load_key()
    f = Fernet(key)
    with open("encrypted_git_token.txt", "rb") as encrypted_file:
        encrypted_token = encrypted_file.read()
    return f.decrypt(encrypted_token).decode()

# Example usage
token = decrypt_token()
if not token:
    raise ValueError("Failed to decrypt the token")


In [5]:
def generate_ssh_key(email, key_path):
    os.makedirs(os.path.dirname(key_path), exist_ok=True)
    subprocess.run(['ssh-keygen', '-t', 'rsa', '-b', '4096', '-C', email, '-f', key_path, '-N', ''])

def read_public_key(key_path):
    with open(f"{key_path}.pub", 'r') as file:
        public_key = file.read()
    return public_key

def add_ssh_key_to_github(token, title, key):
    url = "https://api.github.com/user/keys"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    data = {
        "title": title,
        "key": key
    }
    response = requests.post(url, headers=headers, json=data)
    return response.status_code, response.json()

# # Example usage
# email = 'your_email@example.com'
# key_path = os.path.expanduser('~/.ssh/id_rsa')
# token = "your_github_token"
# title = "My SSH Key"

# generate_ssh_key(email, key_path)
# public_key = read_public_key(key_path)
# status_code, response = add_ssh_key_to_github(token, title, public_key)

# print(status_code, response)


In [6]:
# Example usage
email = ''
try:
    email = os.getenv('USER_EMAIL_ID')
except:
    email = username+"@ext.uber.com"
key_path = os.path.expanduser('~/.ssh/id_rsa')
token = decrypt_token()
title = "LP SSH Key"

In [7]:
generate_ssh_key(email, key_path)

Generating public/private rsa key pair.
Your identification has been saved in /home/mnizam1/.ssh/id_rsa.
Your public key has been saved in /home/mnizam1/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:k+j8/a3tZ88nSl/QI0mCEhF8pD6rhn9w8nHQVvu8z10 mnizam1@ext.uber.com
The key's randomart image is:
+---[RSA 4096]----+
|      .++.       |
|       .o...     |
|       oo.....   |
|      .o.+ .o .. |
|      .oS   oo...|
|     = ooo   o...|
|    . B.o   . . E|
|   . ..+ . . =oo*|
|    oo. . ..++***|
+----[SHA256]-----+


In [8]:
public_key = read_public_key(key_path)
public_key

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDUfH40miksNzrioRUT2jTz4DVaMV4Xbe0XbA+eXCH8ASmnptyh8dPyRkUCn9xSjQalpSzD5eczyM5mfcASMJ5EjpaIVlCmCPL/2D14hz33sOhH1rhaQajNNSIVnS+bprFDz/qybzxZK+15LbFijmJn8nw9+R30HoVrDc8bHaGQvdQ7P0sgswfPFkWersSudzjJwO1ebGg2qHiEeXbKyAgjkDnbJMgFEAJG6495cGweu708KkVVqrzQ+RkQzbA28hvBiBnSFCY/8X5btTQVUiRj5+etB16YH0BpI5nDQEpP0eqlMdlRURsvkDjgSXkw+rxwzZXpoEjwCC44Hiujah6CvREFWXDB8MMrA6zjii1f8WYprFy+U8TaVfra16KVvM3GEcIPhxVuwxBzj9l9FI2p4uAw6vC4j8iNi23a/i/wHk/6DWbU+5RrP7I5MEUULyHAnOYGqNTZAH5JGyUoOF5UCHQV2hnyieYPDQLZ1E5V1eq+XZ3Rh/LAR6qvT/O/uFPcOJwVVtb5gaa1DP9qtix+AC6VmfJ56GqhO/odpoUVRaGc0vBwnk0Lyd9yszQzyhxtUW1HECD0T9R8ltYQFXGMawaDp/zVWz8KHodw+Ap4hy6BvVhRuaSB90mO3IRlnBRxQ7rkfzk0B2ZS2Sd3h4pBrC5XNzbPjn6ZA6ZN2kbvxw== mnizam1@ext.uber.com\n'

In [9]:
status_code, response = add_ssh_key_to_github(token, title, public_key)
status_code, response

(201,
 {'id': 111489645,
  'key': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDUfH40miksNzrioRUT2jTz4DVaMV4Xbe0XbA+eXCH8ASmnptyh8dPyRkUCn9xSjQalpSzD5eczyM5mfcASMJ5EjpaIVlCmCPL/2D14hz33sOhH1rhaQajNNSIVnS+bprFDz/qybzxZK+15LbFijmJn8nw9+R30HoVrDc8bHaGQvdQ7P0sgswfPFkWersSudzjJwO1ebGg2qHiEeXbKyAgjkDnbJMgFEAJG6495cGweu708KkVVqrzQ+RkQzbA28hvBiBnSFCY/8X5btTQVUiRj5+etB16YH0BpI5nDQEpP0eqlMdlRURsvkDjgSXkw+rxwzZXpoEjwCC44Hiujah6CvREFWXDB8MMrA6zjii1f8WYprFy+U8TaVfra16KVvM3GEcIPhxVuwxBzj9l9FI2p4uAw6vC4j8iNi23a/i/wHk/6DWbU+5RrP7I5MEUULyHAnOYGqNTZAH5JGyUoOF5UCHQV2hnyieYPDQLZ1E5V1eq+XZ3Rh/LAR6qvT/O/uFPcOJwVVtb5gaa1DP9qtix+AC6VmfJ56GqhO/odpoUVRaGc0vBwnk0Lyd9yszQzyhxtUW1HECD0T9R8ltYQFXGMawaDp/zVWz8KHodw+Ap4hy6BvVhRuaSB90mO3IRlnBRxQ7rkfzk0B2ZS2Sd3h4pBrC5XNzbPjn6ZA6ZN2kbvxw==',
  'url': 'https://api.github.com/user/keys/111489645',
  'title': 'LP SSH Key',
  'verified': True,
  'created_at': '2024-11-08T01:52:30Z',
  'read_only': False})